In [ ]:
# E-Waste Image Classification using EfficientNetV2B0 (Enhanced Version with Grad-CAM)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetV2B0
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import cv2
import matplotlib.cm as cm
import os
import zipfile

# ✅ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ Unzip dataset
zip_path = "/content/drive/MyDrive/internship/modified-dataset.zip"  # Your dataset zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content")

DATA_DIR = "/content/modified-dataset"
print("✅ Dataset extracted to:", DATA_DIR)

# ✅ Parameters
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

# ✅ Load dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names

# ✅ Prefetch
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# ✅ Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomBrightness(0.2),
])

# ✅ Build Model
base_model = EfficientNetV2B0(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
base_model.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = tf.keras.applications.efficientnet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(len(class_names), activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# ✅ Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=3)
]

# ✅ Initial Training
history = model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=callbacks)

# ✅ Fine-Tuning
base_model.trainable = True
for layer in base_model.layers[:int(len(base_model.layers) * 0.8)]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
fine_tune_history = model.fit(train_ds, validation_data=val_ds, epochs=5, callbacks=callbacks)

# ✅ Evaluation
y_true = np.concatenate([y for _, y in val_ds], axis=0)
y_pred_probs = model.predict(val_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

# ✅ Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()

# ✅ Classification Report
print(classification_report(y_true, y_pred, target_names=class_names))

# ✅ Accuracy Plot
plt.plot(history.history['accuracy'] + fine_tune_history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'] + fine_tune_history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

# ✅ Save Model
model.save("e_waste_classifier_model.keras")
print("✅ Model saved successfully.")

# ✅ Grad-CAM Function
def make_gradcam_heatmap(img_array, model, last_conv_layer_name='block6a_activation', pred_index=None):
    base_model = model.get_layer("efficientnetv2-b0")
    grad_model = tf.keras.models.Model([model.inputs],
                                       [base_model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# ✅ Grad-CAM Visualization
image_path = os.path.join(DATA_DIR, "test/Battery/battery_22.jpg")
img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = tf.keras.applications.efficientnet_v2.preprocess_input(img_array)

heatmap = make_gradcam_heatmap(img_array, model)
img_cv = cv2.imread(image_path)
heatmap_resized = cv2.resize(heatmap, (img_cv.shape[1], img_cv.shape[0]))
heatmap_colored = cm.jet(heatmap_resized)[:, :, :3]
superimposed_img = heatmap_colored * 0.4 + img_cv / 255.0

plt.figure(figsize=(8, 6))
plt.imshow(superimposed_img)
plt.axis('off')
plt.title("Grad-CAM on E-Waste Image (Battery)")
plt.tight_layout()
plt.show()
